In [1]:
!pip install numpy pandas
!pip install scikit-learn

In [2]:
import numpy as np
import pandas as pd
import nltk

#load dataset
movies = pd.read_csv(r'D:\Users\Jupyter notebook\ML PROJECTS\RecommendationSystem\tmdb_5000_movies.csv', encoding = 'unicode_escape')
credits = pd.read_csv(r'D:\Users\Jupyter notebook\ML PROJECTS\RecommendationSystem\tmdb_5000_credits.csv', encoding = 'unicode_escape')

#merge dataset
movies = movies.merge(credits,on = 'title')

#selecting features
movies=  movies[['movie_id', 'title','overview','genres','keywords','cast','crew','vote_count']]

movies.isnull().sum()
movies.dropna(inplace = True)

#converting stringfied to list
# Function to convert stringified list to actual list
import ast
def convert(obj):
    try:
        return [i['name'] for i in ast.literal_eval(obj)]
    except:
        return []

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert)
movies['crew'] = movies['crew'].apply(lambda x: convert(x)[:3])  # Taking top 3 crew members

movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['tags']= movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

new_df = movies[['movie_id','title','tags']]
new_df.loc[:, 'tags'] = new_df['tags'].apply(lambda x: " ".join(x))
new_df.loc[:, 'tags'] = new_df['tags'].apply(lambda x: x.lower())


#stemming done
from nltk.stem.porter import PorterStemmer 
ps = PorterStemmer()
def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))  # Use the global `ps` instance
    
    return " ".join(y)

new_df.loc[:, 'tags'] = new_df['tags'].apply(stem)

#new_df.head()
#vectorisation done
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words= 'english')
vector = cv.fit_transform(new_df['tags']).toarray()

#similarity matrix made
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
sorted(list(enumerate(similarity[0])),reverse = True, key = lambda x : x[1])[1:6]

# recommending movies
def recommend (movie) :
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse = True, key = lambda x : x[1])[1:10]
    #print(f"Movies similar to '{movie_title}':")
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

#example usage
recommend('Avatar')

Aliens vs Predator: Requiem
Titan A.E.
Battle: Los Angeles
Independence Day
Predator
Falcon Rising
Lifeforce
Aliens
U.F.O.


In [4]:
import pickle
pickle.dump(new_df,open('movies.pkl', 'wb'))
pickle.dump(new_df.to_dict(),open('movies_dict.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))